## Cooking with ClarityNLP - Session #5

Today we will also be focusing on **integration of external APIs** using ClarityNLP, so you can leverage cool resources from Google, Amazon, IBM, GitHub, or even other NLP platforms. For background on installing and using ClarityNLP, please see our [documentation](https://claritynlp.readthedocs.io/en/latest/index.html).  We welcome questions via Slack or on [GitHub](https://github.com/ClarityNLP/ClarityNLP/issues).

### Review: Ingesting Documents

In order to run NLP jobs using ClarityNLP, data must first be ingested into the system.  You can ingest data from various sources (eg. flat files, relational databases, APIs, etc) and of various types (eg. txt, doc, pdf, etc). Today we will cover one of the most common ingestion patterns-- bringing in data from a CSV.  ClarityNLP has a user interface to support CSV ingestion.  In a typical instance, this will be located at `localhost:6543/csv`.

![Postman.png](assets/Ingest_UI.png)

The process of ingesting data from CSV involves the following steps:
1. Select your CSV file to load column headers
2. Assign the required fields for ClarityNLP to columns in your file
3. Add any additional fields you would like to include from your source file
4. Start the Import process


### Review: How to Run NLPQL

In order to run NLPQL, you must submit it to a ClarityNLP server either via API or via the ClarityNLP user interface.  If you are running a local instance, the API endpoint is typically `localhost:5000/nlpql`.  NLPQL should be POSTed as text/plain.  An example from [Postman](www.postman.com) is shown below.

![Postman.png](assets/Postman.png)

If you are unfamiliar with using tools such as Postman, you can submit NLPQL via the ClarityNLP user interface running in a web browser. For local instances, this will be at [localhost:8200/runner](localhost:8200/runner). 

![NLPQL_Runner.png](assets/NLPQL_Runner.png)

### Running NLPQL from Jupyter Notebooks
If you wish to run NLPQL directly from this notebook, then please use the following code.  (Note, this assumes an API url of `localhost:5000/`)

In [32]:
# This code below is only required for running ClarityNLP in Jupyter notebooks. It is not required if running NLPQL via API or the ClarityNLP GUI.
import pandas as pd
import claritynlp_notebook_helpers as claritynlp

# Import keys for the APIs used in this session
from keys import *

## Case #1:  Sentiment Analysis
For this  Cooking session, we are going to integrate a few external APIs that perform [sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis) and enable their use within the ClarityNLP ecosystem.  By the end of the session, you should have a good handle on how to incorporate any REST API into your [NLPQL](https://clarity-nlp.readthedocs.io/en/latest/user_guide/intro/overview.html#example-nlpql-phenotype-walkthrough) phenotypes.

### 1.1 Identify external APIs for sentiment analysis

For this example, we want to leverage some of the brilliant minds in text analytics to help us perform Sentiment Analysis using ClarityNLP.  You may or may not be surprised to learn that there are >100 APIs out there for performing sentiment analysis.

![NLPQL_Runner.png](assets/Sentiment_APIs.png)

Our first stop will be [Microsoft Azure Text Analytics](https://westus.dev.cognitive.microsoft.com/docs/services/TextAnalytics.V2.0/operations/56f30ceeeda5650db055a3c9/console).  The Azure Sentiment API lets you pass in a simple sentence or group of sentences and get back an overall sentiment score from 0 to 1.  0 being very negative and 1 very positive.

Here is an example from Postman:

![NLPQL_Runner.png](assets/Azure_Sentiment_Query.png)

The sentiment score for the above sentence is very low (i.e., negative).  Let's try something a little more upbeat.

![NLPQL_Runner.png](assets/Azure_Happy_Query.png)

As you can see, we have a much more positive score (99+).  It's pretty fun to play around with just different sentences ("I am super mad at you" scores a 0.14 whereas "I am not super mad at you" score a 0.03).  Cool stuff, but our goal today is to look at how we might integrate such an API into ClarityNLP. 

### 1.2 Transforming APIs into Custom Tasks 

*Start with a Template*

The first thing we'll do is start with a [Custom API Task Base Template](https://github.com/ClarityNLP/ClarityNLP/blob/ceb40586257078ef4f3f7ea91739141d47e83748/nlp/custom_tasks/SampleAPITask.py). This sample task calls an API to assign a random Chuck Norris joke to every document.

```python
from tasks.task_utilities import BaseTask
from pymongo import MongoClient
import requests


class SampleAPITask(BaseTask):
    task_name = "ChuckNorrisJokeTask"

    # NLPQL

    # define sampleTask:
    # Clarity.ChuckNorrisJokeTask({
    #   documentset: [ProviderNotes]
    # });

    def run_custom_task(self, temp_file, mongo_client: MongoClient):
        for doc in self.docs:

            response = requests.post('http://api.icndb.com/jokes/random')
            if response.status_code == 200:
                json_response = response.json()
                if json_response['type'] == 'success':
                    val = json_response['value']
                    obj = {
                        'joke': val['joke']
                    }

                    # writing results
                    self.write_result_data(temp_file, mongo_client, doc, obj)

            else:
                # writing to log (optional)
                self.write_log_data("OOPS", "No jokes this time!")
```

Now there is a lot of stuff to look at in there, but the only part you really have to pay attention to is the middle part below:

```python
     
        for doc in self.docs:

            response = requests.post('http://api.icndb.com/jokes/random')
            if response.status_code == 200:
                json_response = response.json()
                if json_response['type'] == 'success':
                    val = json_response['value']
                    obj = {
                        'joke': val['joke']
                    }

                    # writing results
                    self.write_result_data(temp_file, mongo_client, doc, obj)

            else:
                # writing to log (optional)
                self.write_log_data("OOPS", "No jokes this time!")
```

What this means is that for each document in the selected documentset, make an API POST request. (The parameter `documentset: [ProviderNotes]` from our NLPQL becomes `self.docs` in the Custom Task code.)  The documentset could be nursing notes containing the word "central line" or  documents tagged "Echocardiogram" or any documentset you can imagine as we discusssed in a [prior Cooking class](https://github.com/ClarityNLP/ClarityNLP/blob/master/notebooks/cooking/Cooking_with_ClarityNLP_091218.ipynb).  They will always be referred to as `self.docs` in a Custom Task.

For every one of these documents, this Task is going to ring up the `http://api.icndb.com/jokes/random` joke API and pick a good joke.  It will then add the joke to an object called `obj` and store it back in our results database.  Now, let's see if we can modify this for our Azure Sentiment API.

*Change the API Call*

For our sentiment analysis, we need to change up the POST headers and body to match the Azure API specifications.  So our we'll change a couple things:

```python
headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': 'XXXXXX'}
payload = {"documents": [{"language": "en", "id": "1", "text": doc}]}
response = requests.post('https://eastus.api.cognitive.microsoft.com/text/analytics/sentiment', headers=headers, json=payload)
```

What we've done is added some of the headers required (like our secret API key) and made the body of the request (the "payload") match the configuration shown in the Postman image above. Then instead of calling the ChuckNorris API, we change our call to Microsoft's URL.

*Change the API Result Handling*

Each API returns results in its own way, so you've got to follow the API documentation so see what you can expect back.  As we saw earlier, this Sentiment API responds with this kind of result:

```json
{
	"documents": [{
		"score": 0.14780092239379883,
		"id": "1"
	}],
	"errors": []
}
```

So we'll build our object a little differently than we did for Chuck Norris.  It'll need to look something like this.

```python
json_response = response.json()
val = json_response['documents'][0]
obj = {
    'sentiment_score': val['score']
    }
```

If we were passing in multiple documents at a time (which we are not), we would need to loop through the response one document at a time.  But in this case, we can just take the first (and only) response, hence the [0].

*API Keys*

Chuck Norris was a free API.  Azure is also free for limited usage, but you need an API key.  In this version, we are going to rely on the user to supply us the API key by passing a parameter in their NLPQL.  Here is example NLPQL we might see:

```
define PatientFeelings:
    Clarity.AzureSentiment({
        documentset: [ProviderNotes],
        "api_key": "{your_api_key}"
    });
```

In order to "catch" this api_key and use it in our Custom Task, we've got a library that get custom_arguments from the NLPQL.  It looks like this:

```
self.pipeline_config.custom_arguments['{parameter_name']
```

So in this case, `self.pipeline_config.custom_arguments['api_key']` would retrieve the API key submitted by the user in the NLPQL.




So putting the whole thing together, we've got our final code:

```python
    for doc in self.docs:
        headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': self.pipeline_config.custom_arguments['api_key']}
        payload = {"documents": [{"language": "en", "id": "1", "text": doc}]}
        response = requests.post('https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0/sentiment', headers=headers, json=payload)
        json_response = response.json()
        val = json_response['documents'][0]
        obj = {
            'sentiment_score': val['score'],
        }

        # writing results
        self.write_result_data(temp_file, mongo_client, doc, obj)

```

To see the final code, with the wrapping back in place and a little bit of error handling thrown in, take a look at [AzureSentimentTask.py](https://github.com/ClarityNLP/ClarityNLP/blob/master/nlp/custom_tasks/AzureSentimentTask.py) in the repo. We made one additional tweak to be sure we are only sending sentences containing birds.  

```java
for doc in self.docs:
  sentence_list = self.get_document_sentences(doc)
    for sentence in sentence_list:
      if any(word.lower() in sentence.lower() for word in self.pipeline_config.terms):
```

### 1.3 Using the Sentiment API Task in a Query 

Our API can now be called using the NLPQL

In [34]:
nlpql ='''
limit 1;

//phenotype name
phenotype "How we feel about birds" version "1";

//include Clarity main NLP libraries
include ClarityCore version "1.0" called Clarity;

termset Birds:
  ["football"];

define BirdFeelings:
  Clarity.AzureSentiment({
    termset:[Birds],
    "api_key":"'''+azure_key+'''"
    });
'''
run_result, main_csv, intermediate_csv, luigi = claritynlp.run_nlpql(nlpql)

Job Successfully Submitted
{
    "intermediate_results_csv": "http://18.220.133.76:5000/job_results/643/phenotype_intermediate",
    "job_id": "643",
    "luigi_task_monitoring": "http://18.220.133.76:8082/static/visualiser/index.html#search__search=job=643",
    "main_results_csv": "http://18.220.133.76:5000/job_results/643/phenotype",
    "phenotype_config": "http://18.220.133.76:5000/phenotype_id/643",
    "phenotype_id": "643",
    "pipeline_configs": [
        "http://18.220.133.76:5000/pipeline_id/862"
    ],
    "pipeline_ids": [
        862
    ],
    "results_viewer": "?job=643",
    "status_endpoint": "http://18.220.133.76:5000/status/643"
}


In [16]:
inter_csv_df = pd.read_csv(intermediate_csv)
inter_csv_df.head()

,_id,batch,concept_code,inserted_date,job_id,nlpql_feature,owner,phenotype_final,pipeline_id,pipeline_type,report_date,report_id,report_type,sentence,sentiment_score,solr_id,source,subject
0,5bcfe705954de200753cb82f,0,-1,2018-10-24 03:29:09.035000,628,BirdFeelings,claritynlp,False,847,AzureSentiment,2132-07-01T00:00:00Z,1660683,Nursing/other,Nursing Note Please see carevue for details Ne...,0.832225,1660683,MIMIC,29607
1,5bcfe705954de200753cb830,0,-1,2018-10-24 03:29:09.181000,628,BirdFeelings,claritynlp,False,847,AzureSentiment,2152-07-14T00:00:00Z,1686791,Nursing/other,A&O x 3 at beginning of shift but seeing birds.,0.500000,1686791,MIMIC,11254
2,5bcfe705954de200753cb831,0,-1,2018-10-24 03:29:09.281000,628,BirdFeelings,claritynlp,False,847,AzureSentiment,2152-07-14T00:00:00Z,1686791,Nursing/other,"o seeing flies, cats and birds.",0.763141,1686791,MIMIC,11254
3,5bcfe705954de200753cb832,0,-1,2018-10-24 03:29:09.441000,628,BirdFeelings,claritynlp,False,847,AzureSentiment,2181-10-04T00:00:00Z,1305517,Nursing/other,"TALKING RAGTIME, VISUAL HALLUCINATIONS- SEES B...",0.957502,1305517,MIMIC,9188
4,5bcfe705954de200753cb833,0,-1,2018-10-24 03:29:09.590000,628,BirdFeelings,claritynlp,False,847,AzureSentiment,2169-02-12T00:00:00Z,2009072,Nursing/other,SEEING BIRDS FLYING IN ROOM..,0.794927,2009072,MIMIC,19632


Finally, add Your new Custom Task to the custom_tasks directory.  It will be automatically incorporated into ClarityNLP and ready to use in NLPQL phenotypes.
![NLPQL_Runner.png](assets/custom_task_folder.png)

### 1.4 Still Feeling Sentimental?  Let's try one more API.


The Azure sentiment analyzer returns a pretty simple response, but maybe even a bit too simple.  Let's try to incorporate one more analyzer, called the [Watson Tone Analyzer](https://tone-analyzer-demo.ng.bluemix.net/?cm_mc_uid=03350901261315399739861&cm_mc_sid_50200000=86620881540314461443&cm_mc_sid_52640000=57530161540314461484) from IBM.  Below of Watson's tone analysis.


![NLPQL_Runner.png](assets/Watson_Tone_Query.png)

As you can see below, Watson provides not a 0 to 1 scale but rather a set of individual tone scales so we can look at its degree of `Sadness` etc

![NLPQL_Runner.png](assets/Watson_Tone_Results.png)

We won't repeat the code development process again, but suffice to say, you want to review the API documentation and modify the ClarityNLP API Base Task to:
- Define the appropriate headers
- Define the appropriate body
- Handle the JSON response

Here's what we came up with, including the boilerplate BaseTask wrapping. 

In [ ]:
from tasks.task_utilities import BaseTask
from pymongo import MongoClient
import requests


class WatsonSentimentTask(BaseTask):
    task_name = "WatsonSentiment"

    # define sampleTask:
    # Clarity.WatsonSentiment({
    #   documentset: [ProviderNotes],
    #   "api_key": "{your_api_key}",
    #   "authorization":"{your_authorization}"
    # });

    def run_custom_task(self, temp_file, mongo_client: MongoClient):
        for doc in self.docs:

            sentence_list = self.get_document_sentences(doc)

            for sentence in sentence_list:
                headers = {'Content-Type': 'application/json', 'apikey': self.pipeline_config.custom_arguments['api_key'], 'authorization': self.pipeline_config.custom_arguments['authorization']}
                payload = {"text": sentence}
                response = requests.post('https://gateway.watsonplatform.net/tone-analyzer/api/v3/tone?version=2017-09-21', headers=headers, json=payload)
                if response.status_code == 200:
                    json_response = response.json()
                    tones = json_response['document_tone']['tones']
                    for tone in tones:
                        obj = {
                            'tone_name': tone['tone_name'],
                            'tone_score': tone['score'],
                            'sentence': sentence
                        }

                        # writing results
                        self.write_result_data(temp_file, mongo_client, doc, obj)

                else:
                    # writing to log (optional)
                    self.write_log_data("OOPS", "No sentiment this time!")

Let's run our example NLPQL

In [36]:
nlpql ='''
limit 1;

//phenotype name
phenotype "How we feel about football" version "1";

//include Clarity main NLP libraries
include ClarityCore version "1.0" called Clarity;

termset Birds:
  ["football"];

define FootballFeelings:
  Clarity.WatsonSentiment({
    termset:[Birds],
    "api_key":"'''+watson_key+'''",
    "authorization":"'''+watson_authorization+'''"
    });
'''
run_result, main_csv, intermediate_csv, luigi = claritynlp.run_nlpql(nlpql)

Job Successfully Submitted
{
    "intermediate_results_csv": "http://18.220.133.76:5000/job_results/645/phenotype_intermediate",
    "job_id": "645",
    "luigi_task_monitoring": "http://18.220.133.76:8082/static/visualiser/index.html#search__search=job=645",
    "main_results_csv": "http://18.220.133.76:5000/job_results/645/phenotype",
    "phenotype_config": "http://18.220.133.76:5000/phenotype_id/645",
    "phenotype_id": "645",
    "pipeline_configs": [
        "http://18.220.133.76:5000/pipeline_id/864"
    ],
    "pipeline_ids": [
        864
    ],
    "results_viewer": "?job=645",
    "status_endpoint": "http://18.220.133.76:5000/status/645"
}


In [23]:
inter_csv_df = pd.read_csv(intermediate_csv)
inter_csv_df.head()

,_id,batch,concept_code,inserted_date,job_id,nlpql_feature,owner,phenotype_final,pipeline_id,pipeline_type,report_date,report_id,report_type,sentence,solr_id,source,subject,tone_name,tone_score
0,5bcfee08954de2015802ced6,0,-1,2018-10-24 03:59:04.593000,632,BirdFeelings,claritynlp,False,851,WatsonSentiment,2156-01-16T00:00:00Z,1888046,Nursing/other,"Reporting being ""down"", misses being at home w...",1888046,MIMIC,8837,Sadness,0.667139
1,5bcfee0c954de2015802ced7,0,-1,2018-10-24 03:59:08.561000,632,BirdFeelings,claritynlp,False,851,WatsonSentiment,2147-08-30T00:00:00Z,515563,Physician,Some exposure to birds (boat).,515563,MIMIC,33824,Anger,0.505060
2,5bcfee0c954de2015802ced8,0,-1,2018-10-24 03:59:08.562000,632,BirdFeelings,claritynlp,False,851,WatsonSentiment,2147-08-30T00:00:00Z,515563,Physician,Some exposure to birds (boat).,515563,MIMIC,33824,Tentative,0.968123
3,5bcfee0d954de2015802ced9,0,-1,2018-10-24 03:59:09.108000,632,BirdFeelings,claritynlp,False,851,WatsonSentiment,2147-08-30T00:00:00Z,515565,Physician,Some exposure to birds (boat).,515565,MIMIC,33824,Anger,0.505060
4,5bcfee0d954de2015802ceda,0,-1,2018-10-24 03:59:09.108000,632,BirdFeelings,claritynlp,False,851,WatsonSentiment,2147-08-30T00:00:00Z,515565,Physician,Some exposure to birds (boat).,515565,MIMIC,33824,Tentative,0.968123


## Task 2. Using APIs in Phenotype Development

Now that we've created a couple custom tasks, let's see how we can use them in creating a phenotype.  For the moment, we will leave aside the notion of whether we trust the sentiment analysis as being accurate (i.e., are birds on boats really angry?).  And let's look for patients who have a **negative sentiment regarding bisphosphonate drugs**.

First to run our basic sentiment analyzer:

In [37]:
nlpql ='''
limit 1;

//phenotype name
phenotype "Bisphosphonate Sentiments" version "1";

//include Clarity main NLP libraries
include ClarityCore version "1.0" called Clarity;

termset Bisphosphonates:
  ["fosamax","alendronate","bisphosphonate"];

define BisphosphonateSentiment:
  Clarity.AzureSentiment({
    termset:[Bisphosphonates],
    "api_key":"'''+azure_key+'''"
    });
'''
run_result, main_csv, intermediate_csv, luigi = claritynlp.run_nlpql(nlpql)

Job Successfully Submitted
{
    "intermediate_results_csv": "http://18.220.133.76:5000/job_results/646/phenotype_intermediate",
    "job_id": "646",
    "luigi_task_monitoring": "http://18.220.133.76:8082/static/visualiser/index.html#search__search=job=646",
    "main_results_csv": "http://18.220.133.76:5000/job_results/646/phenotype",
    "phenotype_config": "http://18.220.133.76:5000/phenotype_id/646",
    "phenotype_id": "646",
    "pipeline_configs": [
        "http://18.220.133.76:5000/pipeline_id/865"
    ],
    "pipeline_ids": [
        865
    ],
    "results_viewer": "?job=646",
    "status_endpoint": "http://18.220.133.76:5000/status/646"
}


In [25]:
inter_csv_df = pd.read_csv(intermediate_csv)
inter_csv_df.head()

,_id,batch,concept_code,inserted_date,job_id,nlpql_feature,owner,phenotype_final,pipeline_id,pipeline_type,report_date,report_id,report_type,sentence,sentiment_score,solr_id,source,subject
0,5bcff786954de202b69c2dff,25,-1,2018-10-24 04:39:34.130000,639,BisphosphonateSentiment,claritynlp,False,858,AzureSentiment,2143-08-29T00:00:00Z,871609,Radiology,[**3267-8-18**] 8:47 AM 1101 BONE DENSITOMETRY...,0.500000,871609,MIMIC,21278
1,5bcff786954de202b99c2dff,0,-1,2018-10-24 04:39:34.155000,639,BisphosphonateSentiment,claritynlp,False,858,AzureSentiment,1999-11-03T00:00:00Z,386478,Clinical Trial Description,We will enroll patients who have undergone car...,0.729338,386478,AACT,NCT00000412
2,5bcff786954de202b99c2e00,0,-1,2018-10-24 04:39:34.400000,639,BisphosphonateSentiment,claritynlp,False,858,AzureSentiment,1999-11-03T00:00:00Z,386478,Clinical Trial Description,We will give Group A active alendronate (10 mg...,0.780525,386478,AACT,NCT00000412
3,5bcff786954de202b69c2e00,25,-1,2018-10-24 04:39:34.412000,639,BisphosphonateSentiment,claritynlp,False,858,AzureSentiment,2116-06-09T00:00:00Z,906769,Radiology,Previously documented demineralization on Fosa...,0.500000,906769,MIMIC,22476
4,5bcff786954de202b99c2e01,0,-1,2018-10-24 04:39:34.502000,639,BisphosphonateSentiment,claritynlp,False,858,AzureSentiment,1999-11-03T00:00:00Z,386478,Clinical Trial Description,We will give Group B placebo alendronate and a...,0.824905,386478,AACT,NCT00000412


Now let's find those with negative sentiments:

In [38]:
nlpql ='''
limit 1;

//phenotype name
phenotype "Bisphosphonate Sentiments" version "1";

//include Clarity main NLP libraries
include ClarityCore version "1.0" called Clarity;

termset Bisphosphonates:
  ["fosamax","alendronate","bisphosphonate"];

define BisphosphonateSentiment:
  Clarity.AzureSentiment({
    termset:[Bisphosphonates],
    "api_key":"'''+azure_key+'''"
    });

context Patient;

define final DownOnBisphonates:
   where BisphosphonateSentiment.sentiment_score<0.4;
'''
run_result, main_csv, intermediate_csv, luigi = claritynlp.run_nlpql(nlpql)

Job Successfully Submitted
{
    "intermediate_results_csv": "http://18.220.133.76:5000/job_results/647/phenotype_intermediate",
    "job_id": "647",
    "luigi_task_monitoring": "http://18.220.133.76:8082/static/visualiser/index.html#search__search=job=647",
    "main_results_csv": "http://18.220.133.76:5000/job_results/647/phenotype",
    "phenotype_config": "http://18.220.133.76:5000/phenotype_id/647",
    "phenotype_id": "647",
    "pipeline_configs": [
        "http://18.220.133.76:5000/pipeline_id/866"
    ],
    "pipeline_ids": [
        866
    ],
    "results_viewer": "?job=647",
    "status_endpoint": "http://18.220.133.76:5000/status/647"
}


In [31]:
main_csv_df = pd.read_csv(main_csv)
main_csv_df.head()

,_id,batch,concept_code,context_type,inserted_date,job_date,job_id,nlpql_feature,orig_id,owner,...,pipeline_type,raw_definition_text,report_date,report_id,report_type,sentence,sentiment_score,solr_id,source,subject
0,5bcff8f0954de20354761473,0,-1,subject,2018-10-24 04:45:23.308000,2018-10-24 04:45:36.662000,641,DownOnBisphonates,5bcff8e3954de20323761474,641,...,AzureSentiment,BisphosphonateSentiment.sentiment_score < 0.4,2000-01-18T00:00:00Z,386045,Clinical Trial Description,Alendronate is used to treat or prevent osteop...,0.303297,386045,AACT,NCT00000427
1,5bcff8f0954de20354761474,0,-1,subject,2018-10-24 04:45:23.617000,2018-10-24 04:45:36.662000,641,DownOnBisphonates,5bcff8e3954de20323761477,641,...,AzureSentiment,BisphosphonateSentiment.sentiment_score < 0.4,2000-01-18T00:00:00Z,386045,Clinical Trial Description,The goal of this extension is to determine wha...,0.092682,386045,AACT,NCT00000427
2,5bcff8f0954de20354761475,0,-1,subject,2018-10-24 04:45:24.871000,2018-10-24 04:45:36.662000,641,DownOnBisphonates,5bcff8e4954de2032376147f,641,...,AzureSentiment,BisphosphonateSentiment.sentiment_score < 0.4,2153-12-05T00:00:00Z,1041011,Radiology,h/o bisphosphonate therapy presents with 5 day...,0.060890,1041011,MIMIC,40031
3,5bcff8f0954de20354761476,0,-1,subject,2018-10-24 04:45:24.973000,2018-10-24 04:45:36.662000,641,DownOnBisphonates,5bcff8e4954de20323761480,641,...,AzureSentiment,BisphosphonateSentiment.sentiment_score < 0.4,2153-12-05T00:00:00Z,1041011,Radiology,65-year-old woman with multiple myeloma and hi...,0.165001,1041011,MIMIC,40031
4,5bcff8f0954de20354761477,25,-1,subject,2018-10-24 04:45:25.563000,2018-10-24 04:45:36.662000,641,DownOnBisphonates,5bcff8e5954de2032076147d,641,...,AzureSentiment,BisphosphonateSentiment.sentiment_score < 0.4,2186-01-21T00:00:00Z,1687785,Nursing/other,"Meds at home: benedryl, fosamax, imuran, lasix...",0.155428,1687785,MIMIC,12108


We could of course join with other unstructured or structured criteria (as previously demonstrated with [OHDSI cohorts](https://github.com/ClarityNLP/ClarityNLP/blob/master/notebooks/cooking/Cooking_with_ClarityNLP_101018.ipynb). Please see prior [Cookings](https://github.com/ClarityNLP/ClarityNLP/tree/master/notebooks/cooking) for examples. 

## Cooking Sneek Peek: NLPQL Editor

We know-- NLPQL isn't too hard to read or copy/tweak, but it is pretty tough to generate *de novo*.  So we've created an editor that helps you build your NLPQL without worrying about a missed semi-colon here or bracket there.  Let's [check it out](https://nlpql-editor.herokuapp.com/demo.html).

![NLPQL_Runner.png](assets/NLPQL_editor.png)

Thank you for joining this week's Cooking with ClarityNLP!  Please send any requests or ideas for future Cooking shows to charity.hilton@gtri.gatech.edu.

Have a great week!